Building a recommendation system that scrapes online comic book aggregators and suggests other relevant titles. 

Plan:
1. PlannerAgent: decomposes the user request (“recommend me titles similar to X”) into steps.
2. CrawlerAgent: uses WebFetchTool + ParseTool (+ a SiteAdapter) to get raw lists (title, genres, rating, summary).
3. ClassifierAgent: normalizes genres, cleans ratings, embeds descriptions
4. RecommenderAgent: Given a title, find top-k similar titles

In [12]:
from pydantic import BaseModel
from typing import List, Optional

ANILIST_API = "https://graphql.anilist.co"

class Book(BaseModel):
    id: int
    title: str
    url: str
    genres: List[str]
    avg_score: Optional[float] = None
    popularity: Optional[int] = None
    desc: Optional[str] = None

In [19]:
from smolagents import tool
import httpx

ANILIST_QUERY = """
query ($page: Int = 1, $perPage: Int = 10, $search: String) {
  Page(page: $page, perPage: $perPage) {
    media(
      type: MANGA
      countryOfOrigin: KR
      search: $search
      sort: POPULARITY_DESC
    ) {
      id
      title { romaji english native }
      genres
      averageScore
      popularity
      siteUrl
      description(asHtml: false)
      format
    }
  }
}
"""

@tool
def search_manhwa(search: str, page: int = 1, per_page: int = 20) -> list[dict]:
    """
    Query AniList for Korean manhwa via GraphQL and return structured results.

    Args:
        search (str): Free-text title filter (can be empty for broad discovery).
        page (int): Page number for pagination.
        per_page (int): Number of results per page.

    Returns:
        list[dict]: Plain-dict representation of manhwa objects
    """

    gql_vars = {
        "page": page,
        "perPage": per_page,
        "search": None if not search.strip() else search
    }

    payload = {"query": ANILIST_QUERY, "variables": gql_vars}

    with httpx.Client(timeout=20) as client:
        r = client.post(ANILIST_API, json=payload)
        r.raise_for_status()
        media = r.json()["data"]["Page"]["media"]

    results = []
    for m in media:
        results.append(
            dict(
                id=m["id"],
                title=m["title"]["english"] or m["title"]["romaji"] or m["title"]["native"],
                url=m["siteUrl"],
                genres=m.get("genres", []),
                average_score=m.get("averageScore"),
                popularity=m.get("popularity"),
                description=m.get("description"),
            )
        )
    return results

In [20]:
BOOK_DB: list[Book] = []

@tool
def upsert_manhwa(items: list[dict]) -> int :
    """
    Insert (naively) a list of Manhwa into memory.

    Args:
        items (list[Manhwa]): Parsed manhwa objects.

    Returns:
        int: Number of manhwas inserted.
    """
    for rec in items:
        BOOK_DB.append(
            Book(**rec)  # convert back to Pydantic for internal use
        )


    return len(items)

@tool
def get_by_genre(genre: str, min_score: int = 70, top_k: int = 10, allow_no_score: bool = True) -> list[Book]:
    """
    Returns up to top_k manhwa matching genre and above a minimum AniList score. 

    Args:
        genre (str): Genre to match (case-insensitive substring).
        min_score (int): Minimum averageScore on AniList (0-100).
        top_k (int): Max number of results.
        allow_no_score (bool): allow unscored manhwa in search.

    Returns:
        list[Manhwa]: Filtered, score-descending list.
    """
    g = genre.lower()
    matches = [m for m in BOOK_DB if any(g in gg.lower() for gg in m.genres) and (m.average_score or 0) >= min_score]

    if allow_no_score:
        no_score = [
            m for m in BOOK_DB
            if any(g in gg.lower() for gg in m.genres)
            and m.average_score is None
        ]
        matches.extend(no_score)


    matches.sort(key=lambda x: (x.avg_score or 0, x.popularity or 0), reverse=True)
    return matches[:top_k]

In [21]:
import os
from smolagents import LiteLLMModel, ToolCallingAgent

model = LiteLLMModel(model_id="gemini/gemini-2.0-flash-lite", api_key=os.getenv("GEMINI_API_KEY"))

fetch_agent = ToolCallingAgent(
    name="fetch_agent",
    description="pulls manhwa from AniList and stores them.",
    tools=[search_manhwa, upsert_manhwa],
    model=model
)

reco_agent = ToolCallingAgent(
    name="recommender_agent",
    description="given a genre (and optional min score), recommends top manhwa",
    tools=[get_by_genre],
    model=model
)

In [23]:
def planner(user_msg: str) -> str:
    user_msg_low = user_msg.lower()
    if "crawl" in user_msg_low or "index" in user_msg_low or "search" in user_msg_low:
        return fetch_agent.run(
            "Use anilist_search_manhwa(search='null') for a broad discovery, page=1, per_page=50. "
            "Then call upsert_manhwa with the results and return how many were inserted."
        )

    return reco_agent.run(
        f"Call get_by_genre with genre inferred from: '{user_msg}'. "
        "If a minimum score is not specified, default to 75. Then present the top 5 cleanly."
    )

In [24]:
print(planner("crawl and index action manhwa"))
print(planner("Recommend me high-score action manhwa"))

╭───────────────────────────────────────────── New run - fetch_agent ─────────────────────────────────────────────╮
│                                                                                                                 │
│ Use anilist_search_manhwa(search='null') for a broad discovery, page=1, per_page=50. Then call upsert_manhwa    │
│ with the results and return how many were inserted.                                                             │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'search_manhwa' with arguments: {'per_page': 50, 'page': 1, 'search': 'null'}                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: |{'id': 132120, 'title': 'Full Volume', 'url': 'https://anilist.co/manga/132120', 'genres': 
|'Comedy', 'Romance', 'Slice of Life'], 'average_score': 73, 'popularity': 4287, 'description': 'Do-Won has a 
problem. His neighbor, Beom, seems awfully rude, but he’s infuriatingly gorgeous. And the completely ridiculous 
cherry on top? Beom turns out to be Do-Won’s all-time favorite camboy, which only adds to the (sexual) frustration.
Beom is a metaphorical pain in Do-Won’s ass, and Do-Won would like to return the favor physically, thank you very 
much. So, Do-Won has a problem—in his pants. The question is, how is he going to solve it? You’ll find out the 
answer if you crank the volume up, and watch very, very closely…\n<br><br>\n(Source: Lezhin)'}, {'id': 34770, 
'title': 'Full House', 'url': 'https://anilist.co/manga/34770', 'genres': |'Comedy', 'Romance'], 'average_score': 
54, 'popularity': 458, 'description': 'Ellie is an aspiring screenwriter in London, far from her home in Korea. 
While her life is a bit lonely, her one comfort is Full House--the estate left to her by her recently deceased 
father and the only tangible memory of him she has left. Meanwhile, world famous actor Ryder Baye is the obsession 
of many a stalker and tabloid reporter. Ellie barely remembers who he is, until the day she is unceremoniously 
evicted. Somehow, Ryder Baye manages to purchase Full House. Ellie will stop at nothing to reclaim her home. Ryder 
vows to stand his ground and hold onto his new purchase. The battle for Full House is fiery, with tempers ranging 
and sparks flying. What are two enemies to do when there is an intense attraction burning just below the surface? 
Ellie and Ryder may discover that their explosive chemistry under the same roof may turn them into a happy 
couple...as long as they can refrain from killing each other!<br><br>\n(Source: Netcomics)'}, {'id': 85508, 
'title': 'In Full Bloom', 'url': 'https://anilist.co/manga/85508', 'genres': |'Drama', 'Psychological'], 
'average_score': None, 'popularity': 142, 'description': "One ruler...One schemer...and...One assassin\r\n\r\nThe 
kingdom of Juushin is suffering at the hands of a cruel emperor. In order to stop his father's rule of tyranny, the
crown prince will enlist the aid of a legendary assassin with reasons of her own for wanting the emperor dead."}, 
{'id': 144445, 'title': 'Blazing Sun', 'url': 'https://anilist.co/manga/144445', 'genres': |'Drama', 'Hentai'], 
'average_score': None, 'popularity': 96, 'description': '“You may be rich, but that doesn’t mean you can just do 
whatever you want with others!” A young man with nothing to lose will do anything to rise to the 
top.<br>\n<br>\n(Source: TOPTOON+)'}, {'id': 137968, 'title': 'Milmyeon Danggigo Danggimyeon Mireora', 'url': 
'https://anilist.co/manga/137968', 'genres': |'Romance'], 'average_score': None, 'popularity': 74, 'description': 
'Rahyeon, a passive student who usually has bad luck, manages to attend a prestigious college. But was it karma? On
the first day of admission he gets involved with a very beautiful but crazy person...<br><br>(Source: Lezhin, 
translated)'}, {'id': 168340, 'title': 'Magnetic Pull', 'url': 'https://anilist.co/manga/168340', 'genres': 
|'Drama', 'Hentai'], 'average_score': None, 'popularity': 71, 'description': "A flock of women is getting pulled 
into this new divorcé's life. Is there a magnetic pull that's at work? Or is Joey's ex-wife jinxing 
him?<br><br>(Source: Toomics)"}, {'id': 166963, 'title': 'Full Circle', 'url': 'https://anilist.co/manga/166963', 
'genres': |'Romance'], 'average_score': None, 'popularity': 32, 'description': 'Yujin is a “normal” teenager who’s 
preparing to retake his college entrance exams. He is not into boys. Nope. At least that’s what he tells himself 
when his classmate, Jun-hyeok, starts hitting on him. But as time goes on, Yujin can’t deny his increasing 
attraction to Jun-hyeok and reluctantly admits that he’s probably as straight

[Step 1: Duration 3.13 seconds| Input tokens: 1,229 | Output tokens: 12]

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'No action manhwa were found with a score of 75 or      │
│ higher.'}                                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: No action manhwa were found with a score of 75 or higher.

Final answer: No action manhwa were found with a score of 75 or higher.

[Step 2: Duration 2.48 seconds| Input tokens: 2,572 | Output tokens: 35]

No action manhwa were found with a score of 75 or higher.
